In [113]:
import os
import sys
import json

from operator import attrgetter

In [160]:
with open("New_Albion_Beautified.json") as o:
    file = json.load(o)

In [145]:
class Message(object):
    mid = 0
    party = None
    content = ''
    author = {
        "aid":"0",
        "sheet":"0",
        "name":""
    }
    tone = ''
    created = ''
    whisperedto = []
    isdescription = False
    isstorytelling = False
    
    def __init__(self, dictionary):
        self.mid = dictionary['mid']
        self.party = dictionary['pid']
        self.content = dictionary['content']
        self.aid = dictionary['aid']
        self.sheet = dictionary['sheet']
        self.name = dictionary['name']
        self.tone = dictionary['tone']
        self.whisperedto = dictionary['whisperedto']
        self.created = dictionary['created']
        self.isdescription = dictionary['isdescription']
        self.isstorytelling = dictionary['isstorytelling']
        
    def read(self):
        values = {
            'mid':self.mid,
            'pid':self.party,
            'content':self.content,
            'aid':self.aid,
            'sheet':self.sheet,
            'name':self.name,
            'tone':self.tone,
            'whisperedto':self.whisperedto,
            'created':self.created,
            'isdescription':self.isdescription,
            'isstorytelling':self.isstorytelling
        }
        return values

def NewMessage(dictionary):
    message = Message(dictionary)
    return message
    
class Roll(object):
    mid = 0
    content = ''
    action = {
        "name":""
    }
    author = {
        "aid":"0",
        "sheet":"0",
        "name":""
    }
    created = ''
    result = []

In [159]:
message = {}
Messages = []
partyid = None
limiter = 0
for party in file: # Each party has a party and a content division. This splits the file up into parties (party/content pairs)
    if isinstance(party, dict): # Each segment of a party/content division is a dictionary
        for key, values in party.items():
            if key == 'party': # Here we get each party's id number
                if values != None:
                    partyid = values['id']
                else:
                    partyid = None
            elif key == 'content': # Here we parse the actual content of the parties. Each 'field' is a message or a roll.
                for field in values:
                    if limiter > 20:
                        break
                    if isinstance(field, dict):
                        message.update({
                            'mid':f"{field.setdefault('id', None)}",
                            'pid':f"{partyid}",
                            'content':f"{field.setdefault('content', None)}",
                            'tone':f"{field.setdefault('tone', None)}", # No tone is already stored as str('None'), so asking if message.tone == None will always be false. Instead check message.tone == "None"
                            'whisperedto':f"{str(field.setdefault('whispered_to', None)).replace('[','').replace(']','')}",
                            'created':f"{field.setdefault('created', None)}",
                            'isdescription':f"{field.setdefault('is_description', False)}",
                            'isstorytelling':f"{field.setdefault('is_storytelling', None)}"
                        })
                        if 'author' in field:
                            message.update({
                                'aid':f"{field['author'].setdefault('id', None)}",
                                'sheet':f"{field['author'].setdefault('sheet', None)}",
                                'name':f"{field['author'].setdefault('name_then', None)}"
                            })
                        else:
                            message.update({
                                'aid':None,
                                'sheet':None,
                                'name':None
                            })
                    m = NewMessage(message)
                    Messages.append(m)
                    limiter += 1
mss = 0
Messages.sort(key=attrgetter('mid'))
while mss < len(Messages):
    r = Messages[mss]
    if r.aid == None:
        print(f"     Narrator (description): {r.content}")
    elif r.isdescription == True:
        print(f"     {r.name} (description): {r.content}")
    elif r.tone != "None":
        print(f"     {r.name} ({r.tone}): {r.content}")
    else:
        print(f"     {r.name}: {r.content}")
    mss += 1

     Aria: Hope I'm not stepping on any toes skills-wise, mate. Regardless, due to that overlap you're probably the only one here Aria is actually going to have some respect for!
     Wrenn Wsunn (amused): Oh boy.
     Narrator (description): As you approach the city of New Albion by land or by sea, you see two buildings before anything else. One is your standard Castle, a massive stone fortress on a cliff giving it a view of both the ocean and the city it protects. The other is a somewhat less massive domed building that seems to be the city's center piece.
     Narrator (description): You join the throng of adventurers headed to the castle and can't help but notice the strange signs of damage. There's the standard fire remnants of course, but one building looks waterlogged and it's neighbors are untouched. Further on a building's pieces hover in an arcane field and several mages are arguing over how to fix it.
     Narrator (description): Then there are the weirdly placed statues. So

In [23]:
type(file)

list